In [1]:
import pandas as pd
from tqdm import tqdm
import obspy
import numpy as np
from obspy.core.utcdatetime import UTCDateTime
import sys
import h5py 
sys.path.append("/home/niyiyu/Research/pnwstore/")

import matplotlib.pyplot as plt
from obspy.core.utcdatetime import UTCDateTime
from pnwstore.catalog import QuakeClient
from pnwstore.station import StationClient

from collections import Counter

In [2]:
quakeclient = QuakeClient("cascadia", "pnwstore")
stationclient = StationClient("cascadia", "pnwstore")
pd.set_option('display.max_columns', None)

## metadata to be added
- station location:
       latitude, longitude, elevation
- trace_P/S_polarity
    - added in mpi_extract_pnwml.py
- trace_[P/S]_onset
    - added in mpi_extract_pnwml.py
    
- trace_SNR
- trace_uncertainty

- source_magnitude_type
    - added in mpi_extract_pnwml.py
- source_magnitude_uncertainty
    - added in mpi_extract_pnwml.py
- source_horizontal_uncertainty
    - added in mpi_extract_pnwml.py

In [25]:
# df = pd.read_csv("../data/metadata.csv")
# df = pd.read_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv")
# df = pd.read_csv("/data/whd01/yiyu_data/PNWML/extract/comcat_metadata.csv")
df = pd.read_csv("../data/extract/proc2022_metadata.csv")
df

,source_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_type,source_depth_km,source_magnitude,source_magnitude_type,source_magnitude_uncertainty,source_depth_uncertainty_km,source_horizontal_uncertainty_km,station_network_code,trace_channel,station_code,station_location_code,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_name,trace_sampling_rate_hz,trace_start_time,trace_S_arrival_sample,trace_P_arrival_sample,trace_S_arrival_uncertainty_s,trace_P_arrival_uncertainty_s,trace_P_polarity,trace_S_onset,trace_P_onset,trace_snr_db
0,uw61890762,2022-12-21T16:14:01.360000Z,46.903,-121.933,earthquake,11.90,0.50,ml,0.093,0.50,0.28,CC,BH,SIFT,NaN,TBA,TBA,TBA,"bucket9$0,:3,:15001",100,2022-12-21T16:13:11.360000Z,5414,5220,0.15,0.06,positive,emergent,impulsive,NaN
1,uw61890762,2022-12-21T16:14:01.360000Z,46.903,-121.933,earthquake,11.90,0.50,ml,0.093,0.50,0.28,CC,BH,PR05,NaN,TBA,TBA,TBA,"bucket5$0,:3,:15001",100,2022-12-21T16:13:11.360000Z,5444,5252,0.15,0.06,positive,emergent,impulsive,NaN
2,uw61890762,2022-12-21T16:14:01.360000Z,46.903,-121.933,earthquake,11.90,0.50,ml,0.093,0.50,0.28,CC,BH,PR01,NaN,TBA,TBA,TBA,"bucket6$0,:3,:15001",100,2022-12-21T16:13:11.360000Z,5452,5242,0.15,0.15,undecidable,emergent,emergent,NaN
3,uw61890762,2022-12-21T16:14:01.360000Z,46.903,-121.933,earthquake,11.90,0.50,ml,0.093,0.50,0.28,CC,BH,VOIT,NaN,TBA,TBA,TBA,"bucket3$0,:3,:15001",100,2022-12-21T16:13:11.360000Z,5430,5238,0.06,0.15,undecidable,impulsive,emergent,NaN
4,uw61890742,2022-12-21T13:31:40.220000Z,46.849,-121.753,earthquake,2.33,0.04,ml,0.214,0.53,0.26,UW,EH,STAR,1.0,TBA,TBA,TBA,"bucket3$1,:3,:15001",100,2022-12-21T13:30:50.220000Z,5177,5105,0.06,0.06,undecidable,impulsive,impulsive,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11747,uw61802032,2022-01-15T01:30:38.770000Z,48.617,-123.032,earthquake,12.41,0.91,md,0.665,0.74,0.39,UW,HH,RNWY,NaN,TBA,TBA,TBA,"bucket9$1196,:3,:15001",100,2022-01-15T01:29:48.770000Z,5410,5212,0.03,0.03,positive,impulsive,impulsive,NaN
11748,uw61802032,2022-01-15T01:30:38.770000Z,48.617,-123.032,earthquake,12.41,0.91,md,0.665,0.74,0.39,UW,HH,OLGA,NaN,TBA,TBA,TBA,"bucket6$1112,:3,:15001",100,2022-01-15T01:29:48.770000Z,5617,5332,0.06,0.03,negative,impulsive,impulsive,NaN
11749,uw61802032,2022-01-15T01:30:38.770000Z,48.617,-123.032,earthquake,12.41,0.91,md,0.665,0.74,0.39,UW,HH,LOPEZ,NaN,TBA,TBA,TBA,"bucket4$1165,:3,:15001",100,2022-01-15T01:29:48.770000Z,5763,5415,0.03,0.03,negative,impulsive,impulsive,NaN
11750,uw61802032,2022-01-15T01:30:38.770000Z,48.617,-123.032,earthquake,12.41,0.91,md,0.665,0.74,0.39,UW,HH,LUMI,NaN,TBA,TBA,TBA,"bucket4$1166,:3,:15001",100,2022-01-15T01:29:48.770000Z,5847,5462,0.06,0.15,undecidable,impulsive,emergent,NaN


## check weird arrival times

In [40]:
df = pd.read_csv("../data/extract/proc2022_metadata.csv")
for idx, i in df.iterrows():
    if i['trace_S_arrival_sample'] < 0 or i['trace_P_arrival_sample'] >15000 or i['trace_P_arrival_sample'] < 0 or i['trace_S_arrival_sample'] >15000:
        print(i['source_id'], i['trace_P_arrival_sample'])
        df = df.drop(axis = 0, index = idx)
df = df.reset_index(drop = True)
df.to_csv("../data/extract/proc2022_metadata.csv", sep = ',', index=False)

## modify magnitude type

In [44]:
idx = df[df['source_magnitude_type'] == 'Md'].index[0]
# print(df.iloc[idx])
# df.iloc[idx, 7] = 'md'
# print(len(df[df['source_magnitude_type'] == 'Md']))
# df.to_csv("../data/metadata.csv", sep = ',', index=False)

IndexError: index 0 is out of bounds for axis 0 with size 0

## calculate SNR

In [26]:
def cal_snr(data, percentile = 98, string = False):
    snrs = []
    for icha in range(3):
        snrs.append(20 * np.log(np.percentile(np.abs(data[icha, ids-100:ids + 200]), percentile)/
              np.percentile(np.abs(data[icha, idp-800:idp]), percentile))/np.log(10))
    if string:
        return "%.3f|%.3f|%.3f" % (snrs[0], snrs[1], snrs[2])
    else:
        return snrs

In [27]:
df = pd.read_csv("../data/extract/proc2022_metadata.csv")

In [28]:
f = h5py.File("../data/extract/proc2022_waveforms.hdf5", "r")
snrs = []
for idx in tqdm(range(len(df))):
    ids = df.iloc[idx]['trace_S_arrival_sample']
    idp = df.iloc[idx]['trace_P_arrival_sample']
    tn = df.iloc[idx]['trace_name']
    bucket, narray = tn.split('$')
    x, y, z = iter([int(i) for i in narray.split(',:')])
    data = f['/data/%s' % bucket][x, :y, :z]
    df.iloc[idx, 28] = cal_snr(data, string = True)
#     for s in cal_snr(data, string = False):
#         if not np.isnan(float(s)):
#             snrs.append(float(s))
f.close()

  0%|          | 0/11752 [00:00<?, ?it/s]/tmp/ipykernel_466538/1895525320.py:4: RuntimeWarning: invalid value encountered in double_scalars
  snrs.append(20 * np.log(np.percentile(np.abs(data[icha, ids-100:ids + 200]), percentile)/
100%|██████████| 11752/11752 [00:13<00:00, 841.96it/s]


In [30]:
df.to_csv("../data/extract/proc2022_metadata.csv", sep = ',', index=False)

## filter out low SNR

In [31]:
df = pd.read_csv("../data/extract/proc2022_metadata.csv")
print(len(df))
for idx, i in df.iterrows():
    snrs = [float(s) for s in i['trace_snr_db'].split('|')]
    for s in snrs:
        if s < -20 or s > 80:
            print("drop!")
            df = df.drop(axis = 0, index = idx)
            break
df = df.reset_index(drop = True)
print(len(df))
df.to_csv("../data/extract/proc2022_metadata.csv", sep = ',', index=False)

11752
11752


## EPH2 EH -> HH

In [29]:
df = pd.read_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv")
df.iloc[df[(df['station_code'] == 'EPH2') & (df['trace_channel'] == 'EH')].index, 12] = "HH"
df.to_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv", sep = ',', index=False)

## UW.GNW.BH

## check location 

In [32]:
df = pd.read_csv("../data/extract/proc2022_metadata.csv")
df['station_location_code'].unique()

array([ nan,   1.,   0.])

In [33]:
df.iloc[df[pd.isna(df['station_location_code'])].index, 14] = '--'
df.iloc[df[df['station_location_code'] == 0.0].index, 14] = "00"
df.iloc[df[df['station_location_code'] == 1.0].index, 14] = "01"
# df.iloc[df[df['station_location_code'] == '10'].index, 14] = "01"
df.to_csv("../data/extract/proc2022_metadata.csv", sep = ',', index=False)

## add lat lon elevation

In [34]:
df = pd.read_csv("../data/extract/proc2022_metadata.csv")

In [38]:
error = []
for idx, i in tqdm(df.iterrows(), total = len(df)):
    if df.iloc[idx, 15] == "TBA":
        network = df.iloc[idx, 11]
        station = df.iloc[idx, 13]
        channel = df.iloc[idx, 12] + "Z"
        time = UTCDateTime(df.iloc[idx, 1])
        result = stationclient.query(network = network, station = station, channel = channel, time = time)
        
        try:
            assert len(result) >= 1
            df.iloc[idx, 15] = result.iloc[0, 5]         # latitude
            df.iloc[idx, 16] = result.iloc[0, 6]         # longitude
            df.iloc[idx, 17] = result.iloc[0, 7]         # elevation
            
        except:
            print(idx)
            error.append([idx, i])

100%|██████████| 11749/11749 [00:00<00:00, 20056.25it/s]


In [36]:
df.drop(index = df[df['station_network_code'] == 'IW'].index, inplace = True)
df.drop(index = df[df['station_network_code'] == 'MB'].index, inplace = True)
df.reset_index(drop = True, inplace = True)

In [7]:
for idx, dx in error:
    if dx['station_code'] == "BES4":
        station = dx['station_code']; channel = dx['trace_channel'] + '?'
        time = UTCDateTime(dx['source_origin_time'])
        lat = stationclient.query(network = "UW", station = station, channel = channel,time = time).iloc[0]['latitude']
        lon = stationclient.query(network = "UW", station = station, channel = channel,time = time).iloc[0]['longitude']
        ele = stationclient.query(network = "UW", station = station, channel = channel,time = time).iloc[0]['elevation']
        df.iloc[idx, 15] = lat        # latitude
        df.iloc[idx, 16] = lon        # longitude
        df.iloc[idx, 17] = ele        # elevation

In [37]:
df.to_csv("../data/extract/proc2022_metadata.csv", sep = ',', index=False)

## merge quarry blast/others into explosion/earthquake

In [14]:
df = pd.read_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv")

In [5]:
df.iloc[df[df['source_type'] == 'quarry blast'].index, 4] = 'explosion'
df.iloc[df[df['source_id'] == 'uw61553011'].index, 4] = 'earthquake'
df.iloc[df[df['source_id'] == 'uw61733307'].index, 4] = 'earthquake'
df.iloc[df[df['source_id'] == 'uw61742182'].index, 4] = 'explosion'

In [6]:
df.iloc[df[df['source_id'] == 'uw61788642'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61587947'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61585656'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61565312'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61166917'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61212391'].index, 4] = 'surface event'

In [7]:
df.iloc[df[df['source_id'] == 'uw61542931'].index, 4] = 'thunder'

In [8]:
df.iloc[df[df['source_id'] == 'uw61500398'].index, 4] = 'low-frequency earthquake'

In [51]:
df.to_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv", sep = ',', index=False)

## remove event of sonic bloom & low-frequency earthquake

In [34]:
df = pd.read_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv")

In [9]:
df.drop(axis = 1, index = df[df['source_type']=='sonic boom'].index, inplace = True)
df.drop(axis = 1, index = df[df['source_type']=='low-frequency earthquake'].index, inplace = True)
df.drop(axis = 1, index = df[df['source_type']=='surface event'].index, inplace = True)
df.drop(axis = 1, index = df[df['source_type']=='thunder'].index, inplace = True)

In [10]:
df['source_type'].unique()

array(['earthquake', 'explosion'], dtype=object)

In [11]:
df = df.reset_index(drop = True)

In [11]:
df.to_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv", sep = ',', index=False)

## drop NV

In [12]:
df = pd.read_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv")

In [13]:
df.drop(axis = 1, index = df[df['station_network_code'] == 'NV'].index, inplace = True)
df = df.reset_index(drop = True)

In [14]:
df.to_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv", sep = ',', index=False)

## drop duplicated events

In [4]:
df = pd.read_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv")

In [66]:
df[(df['station_code'] == 'YA2') & (df['trace_channel'] == 'EH') & (df['event_id'] == 'uw10558538')]

,event_id,source_origin_time,source_latitude_deg,source_longitude_deg,event_type,source_depth_km,source_magnitude,source_magnitude_type,source_magnitude_uncertainty,source_depth_uncertainty_km,source_horizontal_uncertainty_km,station_network_code,trace_channel,station_code,station_location_code,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_name,trace_sampling_rate_hz,trace_start_time,trace_S_arrival_sample,trace_P_arrival_sample,trace_S_arrival_uncertainty_s,trace_P_arrival_uncertainty_s,trace_P_polarity,trace_S_onset,trace_P_onset,trace_snr_db,split


In [65]:
2691, 1482, 1917, 1055, 635, 512, 1594, 998, 633, 1347, 
1736, 1060, 1051, 1910, 667, 1343, 1545, 1720, 2791, 2725, 
2723, 2720, 2713, 2701

(2723, 2720, 2713, 2701)

In [64]:
df.drop(axis = 1, index = 2701, inplace = True)
df.reset_index(drop = True, inplace = True)

## azimuth

In [2]:
df = pd.read_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv")
df['year'] = df.apply(lambda row: row['source_origin_time'].split("-")[0], axis = 1)

In [3]:
df

,event_id,source_origin_time,source_latitude_deg,source_longitude_deg,event_type,source_depth_km,source_magnitude,source_magnitude_type,source_magnitude_uncertainty,source_depth_uncertainty_km,...,trace_S_arrival_sample,trace_P_arrival_sample,trace_S_arrival_uncertainty_s,trace_P_arrival_uncertainty_s,trace_P_polarity,trace_S_onset,trace_P_onset,trace_snr_db,split,year
0,uw10564613,2002-10-03T01:56:49.530000Z,48.553,-122.520,earthquake,14.907,2.10,md,0.030,1.68,...,8097,6733,0.04,0.02,undecidable,NaN,NaN,6.135|3.065|11.766,train,2002
1,uw10564613,2002-10-03T01:56:49.530000Z,48.553,-122.520,earthquake,14.907,2.10,md,0.030,1.68,...,7258,6238,0.04,0.01,undecidable,NaN,NaN,nan|nan|23.497,train,2002
2,uw10564613,2002-10-03T01:56:49.530000Z,48.553,-122.520,earthquake,14.907,2.10,md,0.030,1.68,...,7037,6113,0.08,0.04,undecidable,NaN,NaN,1.756|3.057|3.551,test,2002
3,uw10564613,2002-10-03T01:56:49.530000Z,48.553,-122.520,earthquake,14.907,2.10,md,0.030,1.68,...,5894,5442,0.04,0.01,negative,NaN,NaN,nan|nan|26.336,train,2002
4,uw10568748,2002-09-26T07:00:04.860000Z,48.481,-123.133,earthquake,22.748,2.90,md,0.030,0.91,...,8647,6948,0.04,0.01,negative,NaN,NaN,10.881|17.107|2.242,dev,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156545,uw61722006,2021-04-02T10:48:38.940000Z,45.743,-122.621,earthquake,5.610,1.18,ml,0.105,0.69,...,6527,5836,0.06,0.06,negative,impulsive,impulsive,-2.635|0.701|-4.224,test,2021
156546,uw61722006,2021-04-02T10:48:38.940000Z,45.743,-122.621,earthquake,5.610,1.18,ml,0.105,0.69,...,6622,5896,0.15,0.06,negative,emergent,impulsive,13.674|11.735|10.436,train,2021
156547,uw61722006,2021-04-02T10:48:38.940000Z,45.743,-122.621,earthquake,5.610,1.18,ml,0.105,0.69,...,6923,6076,0.15,0.15,undecidable,emergent,emergent,-0.268|4.398|7.123,train,2021
156548,uw61722006,2021-04-02T10:48:38.940000Z,45.743,-122.621,earthquake,5.610,1.18,ml,0.105,0.69,...,6870,6049,0.15,0.15,undecidable,emergent,emergent,14.683|12.908|0.180,train,2021


In [4]:
for eid in tqdm(df['event_id'].unique()):
    dftmp = df[df['event_id'] == eid]
    e = obspy.read_events(f"/1-fnp/pnwstore1/p-wd11/PNWQuakeML/libcomcat/{dftmp['year'].unique()[0]}/{eid}.xml")[0]

  0%|          | 27/42677 [00:02<1:17:03,  9.23it/s]


KeyboardInterrupt: 

## drop wrong picks

In [5]:
df.drop(axis = 1, index = df[df['trace_S_arrival_sample']<= df['trace_P_arrival_sample']].index, inplace = True)
df = df.reset_index(drop = True)
df.to_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv", sep = ',', index=False)